<a href="https://colab.research.google.com/github/DLSeed/DeepLearning/blob/main/sovits_f0%E4%B8%80%E9%94%AE%E8%AE%AD%E7%BB%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[sovits合集导航](https://github.com/IceKyrin/sovits_guide)**

**本colab为Rcell版sovits2.0(f0分支)，不支持单人模型，模型仅在三个一键脚本内部互通**

可以使用**预模型（已替换为22050hz）**节省训练时间，id范围0-7，**最多八个人物**，按提示操作即可

[sovits_f0](https://github.com/innnky/so-vits-svc)

**默认每隔2000次step保存一次，可在“每隔多少次step保存一次断点”部分进行修改。**

**在看到进度save之前不要轻易退出，以免丢失进度。**

在此基础上修改为sovits配置，配合sovits一键制作数据集使用
**为达最佳效果，建议下载公开歌声数据集opencpop，进行多人模型训练**

**格式参考vits专栏三件套（评论区）**[vits注解](https://www.bilibili.com/read/cv18478187)

95%的问题都可以参考专栏解决，剩下的我也不会了

前置：[一键制作数据集](https://colab.research.google.com/drive/1avWZ_N5BsQcq45XkwQkDpmp912CLZS0n#scrollTo=xx2oAf90btEy)

后置：[一键合成](https://colab.research.google.com/drive/1F3VpHCi9eridGw1F1hbqR7qhXGKuSCus#scrollTo=vjkgBV7j2cVJ)




In [1]:
# 查看显卡
!nvidia-smi

Fri Oct 21 03:06:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title 准备
#@markdown 定义工具函数 `run_command` `run_command_by_line` `get_symbols` 和 `get_tensorboard_showing`
# forked from https://www.endpointdev.com/blog/2015/01/getting-realtime-output-using-python/
import os
import subprocess
def run_command(command_args):
    def print_pipe(raw):
        return print(raw.decode("utf-8"), end='')
    try:
      process = subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      out, err = process.communicate()
    except:
      pass
    print_pipe(out)
    print_pipe(err)
    rc = process.poll()
    return rc

def run_command_by_line(command_args):
    def print_pipe(raw):
        return print(raw.decode("utf-8"), end='')
    with subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE) as process:
      while process.poll() is None:
        print_pipe(process.stdout.readline())
      [print_pipe(line) for line in process.stderr.readlines()]
    return

def get_tensorboard_showing(logdir):
    from multiprocessing import Process
    from tensorboard import notebook
    import tensorflow as tf
    import time

    def run_tb():
        run_command_by_line(["tensorboard","--reload_interval", "30",  "--logdir", logdir, "--bind_all"])
    
    def monitor_tb():
        while True:
            try:
                notebook.display(height=998)
                break
            except Exception as e:
                print(e)
                time.sleep(3)

    if param_enable_tb:
        Process(target=run_tb).start()
        Process(target=monitor_tb).start()

In [3]:
#@title 下载依赖库
#@markdown 取消勾选则不会节省空间
colab_save_space = True #@param {type:"boolean"}
os.chdir('/content')
run_command_by_line(["git", "clone", "https://github.com/DLSeed/sovits_f0_train.git", "-b", "main" if colab_save_space else "main"])
os.chdir('/content/sovits_f0_train')
!pip install -r requirements.txt
!sudo apt-get install espeak -y
!sudo apt-get install p7zip-full p7zip-rar
!pip install demjson

Cloning into 'sovits_f0_train'...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 13.9 MB/s 
     |████████████████████████████████| 183 kB 71.1 MB/s 
     |████████████████████████████████| 11.6 MB 56.8 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 49 kB 6.1 MB/s 
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
     |████████████████████████████████| 6.8 MB 48.4 MB/s 
     |████████████████████████████████| 238 kB 63.7 MB/s 
     |████████████████████████████████| 72 kB 1.7 MB/s 
     |████████████████████████████████| 197 kB 67.5 MB/s 
     |████████████████████████████████| 56 kB 5.5 MB/s 
     |████████████████████████████████| 500 kB 70.2 MB/s 
     |████████████████████████████████| 41 kB 659 kB/s 
     |████████████████████████████████| 210 kB 76.1 MB/s 
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-a

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 27 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 

In [4]:
#@title 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#@title 解压数据集
!mkdir /content/sovits_f0_train/dataset

#@markdown **数据集一键制作出的压缩包，名称完全相同**

#@markdown 每次解压**一个**，然后确认完成后**修改名称**、**依次**解压**所有**压缩包

#@markdown 压缩包名称(不带zip)
DATASETNAME = "out_lamianer"  #@param {type:"string"}
#@markdown 压缩包路径(这个名字是接着上一篇的输出zip)
ZIP_PATH = "/content/drive/MyDrive/dataset/out_lamianer.zip"  #@param {type:"string"}
DATASETPATH = "/content/sovits_f0_train/" + DATASETNAME
%cd /content/sovits_f0_train/dataset
!cp {ZIP_PATH} {DATASETNAME}.zip
!unzip -q {DATASETNAME}.zip

/content/sovits_f0_train/dataset


In [6]:
#@title 生成filelist
import re
import demjson
!mkdir /content/sovits_f0_train/filelist
%cd /content/sovits_f0_train/


def replace_id(f_list, sp_id):
    if re.search('wav[|](\d+)[|]', f_list[0]):
        split_char = "wav|%s|" % re.search('wav[|](\d+)[|]', f_list[0]).group(1)
    else:
        split_char = "wav|"
    f_list = [x.replace(split_char, f"wav|{sp_id}|") for x in f_list]
    return f_list


# data文件夹下放，各个 工程名/wavs/train(val)(这层可以不要)/xxx.wav
# 自动读取每个工程文件夹下的train(val).txt(xxx.wav|无符号文本)
#@markdown 默认为0
pre = 0  #@param {type:"string"} 
dataset_path = "./dataset"  # 数据合集目录
file_train_name = "train.txt"
file_val_name = "val.txt"
#@markdown 人物名(**必须与数据集制作时人物名相同，个数也相同，英文逗号隔开**)可以单人

#@markdown 就是已经在dataset文件夹下解压出的数据文件夹

#@markdown id即为此处人物排序，从0自动开始计算
speakers = "lamianer" #@param {type:"string"}
speakers = [speaker.strip() for speaker in speakers.split(",")]

print(speakers)

train_f = open("./filelist/train.txt", "w", encoding="utf-8")
val_f = open("./filelist/val.txt", "w", encoding="utf-8")
pre = int(pre)
for i in range(pre, len(speakers) + pre):
    with open(f"{dataset_path}/{speakers[i]}/{file_train_name}", "r", encoding="utf-8") as f:
        file_list = f.readlines()
        train_f.writelines(replace_id(file_list, i))
    with open(f"{dataset_path}/{speakers[i]}/{file_val_name}", "r", encoding="utf-8") as f:
        file_list = f.readlines()
        val_f.writelines(replace_id(file_list, i))
train_f.close()
val_f.close()


/content/sovits_f0_train
['lamianer']


In [7]:
#@title 生成配置文件 在configs下，记得自己下载保留一份，下次用
#@markdown 本colab内可选下载的**预模型已替换为22050hz**，id为0-7可用

#@markdown **预模型请直接使用下一步下载的nyarumul.json，id0-7可用，json无需修改**

#@markdown **预模型的json可以不改动，只放一个id的数据进去**

# forked from https://github.com/CjangCjengh/vits/blob/main/configs/japanese_ss_base2.json

#@markdown 44100采样率，训练时长要求高
high_sample_rate = False #@param {type:"boolean"}
#@markdown 配置文件名称
json_filename = "lamianer.json" #@param {type:"string"}
#@markdown 训练次数
hparams_epochs = 6000 #@param {type:"integer"}
#@markdown 每隔多少次step保存一次断点
hparams_eval_interval = 2000 #@param {type:"integer"}
#@markdown 单次step的文件数（建议在16以内，若使用44100采样率，在8以内）
hparams_batch_size = 16 #@param {type:"integer"}
#@markdown 训练集文件列表
hparams_training_files = "/content/sovits_f0_train/filelist/train.txt" #@param {type:"string"}
#@markdown 验证集文件列表
hparams_validation_files = "/content/sovits_f0_train/filelist/val.txt"#@param {type:"string"}
#@markdown 人物名，多个人物用英文逗号隔开，必须与上面填写的相同
hparams_speaker = "lamianer" #@param {type:"string"}
#@markdown 模型名
hparams_model_name = "nyarumul" #@param {type:"string"}

speakers = [speaker.strip() for speaker in hparams_speaker.split(",")]
print("speakers: ")
for i, speaker in enumerate(speakers):
  print("\t{a}: {b}".format(a=i, b=speaker))
training_json = {
  "train": {
    "log_interval": 200,
    "eval_interval": hparams_eval_interval,
    "seed": 1234 ,
    "epochs": hparams_epochs,
    "learning_rate": 2e-4,
    "betas": [0.8, 0.99],
    "eps": 1e-9,
    "batch_size": hparams_batch_size,
    "fp16_run": True,
    "lr_decay": 0.999875,
    "segment_size": 16384 if high_sample_rate else 8192,
    "init_lr_ratio": 1,
    "warmup_epochs": 0,
    "c_mel": 45,
    "c_kl": 1.0
  },
  "data": {
    "training_files": hparams_training_files,
    "validation_files": hparams_validation_files,
    "text_cleaners":["english_cleaners2"],
    "max_wav_value": 32768.0,
    "sampling_rate": 44100 if high_sample_rate else 22050,
    "filter_length": 2048 if high_sample_rate else 1024,
    "hop_length": 512 if high_sample_rate else 256,
    "win_length": 2048 if high_sample_rate else 1024,
    "n_mel_channels": 128 if high_sample_rate else 80,
    "mel_fmin": 0.0,
    "mel_fmax": None,
    "add_blank": True,
    "n_speakers": len(speakers) if len(speakers) > 1 else 2
  },
  "model": {
    "inter_channels": 192,
    "hidden_channels": 256,
    "filter_channels": 768,
    "n_heads": 2,
    "n_layers": 6,
    "kernel_size": 3,
    "p_dropout": 0.1,
    "resblock": "1",
    "resblock_kernel_sizes": [3,7,11],
    "resblock_dilation_sizes": [[1,3,5], [1,3,5], [1,3,5]],
    "upsample_rates": [8,8,4,2] if high_sample_rate else [8,8,2,2],
    "upsample_initial_channel": 512,
    "upsample_kernel_sizes": [16,16,4,4],
    "n_layers_q": 3,
    "use_spectral_norm": False
  },
  "speakers": speakers
}

if len(speakers) > 1:
  training_json["model"]["gin_channels"] = 256

import demjson
os.chdir('/content/sovits_f0_train/configs')
training_json_text = demjson.encode(training_json)
with open(json_filename, "w") as file:
  file.write(training_json_text)

os.chdir('/content/sovits_f0_train')



speakers: 
	0: lamianer


In [8]:
#@title 预处理
os.chdir('/content/sovits_f0_train/monotonic_align')
!python setup.py build_ext --inplace
os.chdir('/content/sovits_f0_train')

Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/sovits_f0_train/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'monotonic_align.core' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c core.c -o build/temp.linux-x86_64-3.7/core.o
creating /content/sovits_f0_train/monotonic_align/monotonic_align
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymboli

若**使用预模型**（2.0只能用2.0的预模型），直接在预模型基础上训练，节省时间

使用预模型读取pth文件后，文件名的G_steps.pth，steps会乱是正常现象（使用v100训练，batch_size为32，colab显存不够改成了16，所以会乱掉，保存最新生成的模型即可）

程序读取一次预模型后，以后断点恢复训练、用**最新生成的一组pth继续训练**

In [ ]:
#@markdown 22050hz预模型,id为0-7可用
!mkdir /content/drive/MyDrive/nyarumul/
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  !wget https://huggingface.co/spaces/xiaolang/sovits_f0/resolve/main/G_50000.pth -O /content/drive/MyDrive/nyarumul/G_50000.pth
  !wget https://huggingface.co/spaces/xiaolang/sovits_f0/resolve/main/D_50000.pth -O /content/drive/MyDrive/nyarumul/D_50000.pth

--2022-10-09 00:53:02--  https://huggingface.co/spaces/xiaolang/sovits_f0/resolve/main/G_50000.pth
Resolving huggingface.co (huggingface.co)... 54.210.225.113, 54.173.5.192, 52.5.54.249, ...
Connecting to huggingface.co (huggingface.co)|54.210.225.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/89/cb/89cb9b81ebb299ff59bb93f7dd9c2ff81373001bb318b1e3d4df28ab738185bd/531507463f55c4469b4a8f603f6cc61d44a4c060e3dd89b0c4618fffc4b79441?response-content-disposition=attachment%3B%20filename%3D%22G_50000.pth%22&Expires=1665535982&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzg5L2NiLzg5Y2I5YjgxZWJiMjk5ZmY1OWJiOTNmN2RkOWMyZmY4MTM3MzAwMWJiMzE4YjFlM2Q0ZGYyOGFiNzM4MTg1YmQvNTMxNTA3NDYzZjU1YzQ0NjliNGE4ZjYwM2Y2Y2M2MWQ0NGE0YzA2MGUzZGQ4OWIwYzQ2MThmZmZjNGI3OTQ0MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPWF0dGFjaG1lbnQlM0IlMjBmaWxlbmFtZSUzRCUyMkdfNTAwMDAucHRoJTIyIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNz

In [ ]:
#@title 训练

#@markdown 启用tensorboard可视化数据，首次运行可能十分钟左右出现数据

#@markdown epoch次数看/content/drive/MyDrive/数据集名/train.log

#@markdown 数据集名（使用预模型则为nyarumul）
hparams_model_name = "nyarumul"  # @param {type:"string"}
#@markdown 模型自动保存到 /云盘路径/模型名 ,记得定期清理空间！
param_enable_tb = True  # @param {type:"boolean"}
if param_enable_tb:
  #@markdown 云盘路径（一般不改） （定时清云盘！！！pth里保留最新的俩）
  logdir = "/content/drive/MyDrive/"  # @param {type:"string"}
  new_pth_dir = os.path.join(logdir, hparams_model_name)
  get_tensorboard_showing(new_pth_dir)
os.chdir('/content/sovits_f0_train')
#@markdown 配置文件json名（使用预模型则为nyarumul.json）
json_filename = "nyarumul.json"  # @param {type:"string"}
# 这里魔改过until.py的args
run_command_by_line(["python", "train_ms.py", "-c", "configs/{json}".format(json=json_filename), "-m", hparams_model_name, "-l",logdir])

In [10]:
#@title 精简模型
import torch
#@markdown 模型名，数字是最新的模型steps
new_model_name = "G_16000.pth"  # @param {type:"string"}
checkpoint_dict = torch.load(f"{new_pth_dir}/{new_model_name}")
iteration = checkpoint_dict['iteration']
learning_rate = checkpoint_dict['learning_rate']
optimizer = checkpoint_dict['optimizer']
saved_state_dict = checkpoint_dict['model']
print(iteration)
#@markdown 输出xxx_epoch.pth
torch.save({'model': saved_state_dict,
  'iteration': None,
  'optimizer': None,
  'learning_rate': None}, f'{new_pth_dir}/{iteration}_epochs.pth')

321
